# Find unbound states to pair with the bound states

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties
running on node: dig159


### Set working directory to the root of the crispy_shifty repo
TODO set to projects dir

In [2]:
os.chdir("/home/pleung/projects/crispy_shifty")
# os.chdir("/projects/crispy_shifty")

### Fix the paths
Necessary because we copied these designs from perlmutter

In [3]:
from crispy_shifty.utils.io import fix_path_prefixes

folded = os.path.join(
    os.getcwd(), "projects/crispy_shifties/03_fold_bound_states/folded_states.list"
)
new_folded = fix_path_prefixes(
    find="/pscratch/sd/p/pleung",
    replace="/mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties",
    file=folded,
    overwrite=True,
)

https://docs.anaconda.com/anaconda/install



/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### Pair the designed bound states
TODO

In [ ]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "04_pair_bound_states"
design_list_file = os.path.join(
    os.getcwd(),
    "projects/crispy_shifties/03_fold_bound_states/folded_states.list",
)
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

options = " ".join(
    [
        "out:level 200",
        "corrections::beta_nov16 true",
        "indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
    ]
)

extra_kwargs = {
    "reference_csv": "/mnt/home/pleung/projects/crispy_shifty/scaffolds/02_make_free_states/free_state_0s.csv",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.states.pair_bound_state",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="4G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

In [4]:
# !sbatch -a 1-$(cat /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/02_mpnn_bound_states/tasks.cmds | wc -l) /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/02_mpnn_bound_states/run.sh

### Collect scorefiles of designed bound states and concatenate
TODO change to projects dir

In [5]:
# sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
# from crispy_shifty.utils.io import collect_score_file

# simulation_name = "04_pair_bound_states"
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# if not os.path.exists(os.path.join(output_path, "scores.json")):
#     collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile
TODO change to projects dir

In [6]:
# sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
# from crispy_shifty.utils.io import parse_scorefile_linear

# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
# scores_df = scores_df.convert_dtypes()

### Setup for plotting

In [7]:
# sns.set(
#     context="talk",
#     font_scale=1,  # make the font larger; default is pretty small
#     style="ticks",  # make the background white with black lines
#     palette="colorblind",  # a color palette that is colorblind friendly!
# )

### Data exploration
Gonna remove the Rosetta sfxn scoreterms for now

In [8]:
# from crispy_shifty.protocols.design import beta_nov16_terms

# scores_df = scores_df[
#     [term for term in scores_df.columns if term not in beta_nov16_terms]
# ]
# print(len(scores_df))

### Save a list of outputs

In [9]:
# simulation_name = "04_pair_bound_states"
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# with open(os.path.join(output_path, "paired_states.list"), "w") as f:
#     for path in tqdm(scores_df.index):
#         print(path, file=f)

### Prototyping blocks

test `pair_bound_state`

In [4]:
%%time 
import pyrosetta

pyrosetta.init("-corrections::beta_nov16 -indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5")

sys.path.insert(0, "~/projects/crispy_shifty/") # TODO projects
from crispy_shifty.protocols.states import pair_bound_state

t = pair_bound_state(
        None,
        **{
            'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/03_fold_bound_states/decoys/0026/03_fold_bound_states_ef5016a6c9d3470b97c4ecd1ba960f35.pdb.bz2',
            'reference_csv': '/mnt/home/pleung/projects/crispy_shifty/scaffolds/02_make_free_states/free_state_0s.csv',
        }
)
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{i}.pdb")

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.50+release.4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 2021-12-16T00:25:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release r306 2021.50+release.4ff291e 4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 http://www.pyrosetta.org 2021-12-16T00:25:15
core.init: command: PyRosetta -corrections::beta_nov16 -indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5 -database /projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1549180544 seed_offset=0 real_seed=-154918

In [5]:
d = dict(tppose.pose.scores)
from crispy_shifty.protocols.design import beta_nov16_terms

d = {k: v for k, v in d.items() if k not in beta_nov16_terms}

In [6]:
d

{'bb_clash': '250.36671447753906',
 'best_average_DAN_plddts': '0.9204101562',
 'best_average_plddts': '94.379172804',
 'best_model': '2',
 'best_ptm': '0.8931488654',
 'best_rmsd_to_input': '1.0943584816',
 'buns_parent': '2.0',
 'cms_AcB': '297.3140563964844',
 'cms_AnAc': '226.63072204589844',
 'cms_AnAcB': '640.595947265625',
 'cms_AnB': '347.1438293457031',
 'designed_by': 'mpnn',
 'docked_helix': '5',
 'dssp': 'LHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLLLLHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHL',
 'elapsed_time': '5.834771156311035',
 'exposed_hydrophobics_parent': '7.770049571990967',
 'geometry_parent': '1.0',
 'holes_all_parent': '-1.5947035551071167',
 'holes_core_parent': '-2.432673215866089',
 'loop_dist': '11.541058243150694',
 'loop_sc': '0.7584340187061882',
 'looped_length': '171.0',
 'mean_pae': '4.042686462402344',
 'mean_pae_interaction': '4.7735304